<a href="https://colab.research.google.com/github/OliverC2003/Human-Fall-Detection/blob/Week_3/HFD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [25]:
data_folder = r"/content/drive/MyDrive/HFD_files"

In [26]:
example_features = r"/content/drive/MyDrive/HFD_files/video (30)_features.csv"
example_frames = r"/content/drive/MyDrive/HFD_files/video (30).txt"

In [27]:
df_features = pd.read_csv(example_features)

In [28]:
# Step 1: Read the text file
with open(example_frames, 'r') as file:
    data = file.readlines()

# Step 2: Extract the first two values (assuming they are the first two lines)
frame1 = int(data[0].strip())  # First frame
frame2 = int(data[1].strip())  # Second frame

In [29]:
print(frame1, frame2)

80 104


In [30]:
df_features['is_falling'] = df_features['frame'].apply(lambda x: 1 if x in range(frame1, frame2 + 1) else 0)

In [31]:
df_features.head()

,frame,NOSE_x,NOSE_y,LEFT_SHOULDER_x,LEFT_SHOULDER_y,RIGHT_SHOULDER_x,RIGHT_SHOULDER_y,LEFT_HIP_x,LEFT_HIP_y,RIGHT_HIP_x,...,LEFT_KNEE_y,LEFT_ANKLE_x,LEFT_ANKLE_y,RIGHT_ANKLE_x,RIGHT_ANKLE_y,RIGHT_INDEX_x,RIGHT_INDEX_y,LEFT_INDEX_x,LEFT_INDEX_y,is_falling
0,1,0.282740,0.497902,0.144711,0.648973,0.254294,0.681313,0.187293,0.998540,0.238275,...,1.138842,0.221590,1.342223,0.224236,1.405415,0.255230,0.922899,0.161513,0.825024,0
1,2,0.289449,0.490814,0.144357,0.637467,0.243563,0.669886,0.202648,1.002155,0.239115,...,1.180518,0.276934,1.401429,0.240391,1.462306,0.287448,0.949506,0.166210,0.854022,0
2,3,0.300938,0.483526,0.147022,0.627900,0.234477,0.651838,0.216656,1.002364,0.239124,...,1.197397,0.300542,1.421605,0.241697,1.478704,0.292595,0.939510,0.189474,0.873817,0
3,4,0.313169,0.482957,0.152049,0.620945,0.239411,0.652913,0.219518,1.008155,0.240528,...,1.232566,0.298864,1.472134,0.243770,1.555962,0.337951,1.030983,0.187234,0.882367,0
4,5,0.322819,0.482779,0.151722,0.618120,0.264200,0.655914,0.199364,1.008331,0.254126,...,1.225737,0.296572,1.468516,0.262324,1.556030,0.340625,1.048279,0.185746,0.738046,0


In [32]:
df_features[df_features["frame"] == 90]

,frame,NOSE_x,NOSE_y,LEFT_SHOULDER_x,LEFT_SHOULDER_y,RIGHT_SHOULDER_x,RIGHT_SHOULDER_y,LEFT_HIP_x,LEFT_HIP_y,RIGHT_HIP_x,...,LEFT_KNEE_y,LEFT_ANKLE_x,LEFT_ANKLE_y,RIGHT_ANKLE_x,RIGHT_ANKLE_y,RIGHT_INDEX_x,RIGHT_INDEX_y,LEFT_INDEX_x,LEFT_INDEX_y,is_falling
89,90,0.570163,0.632725,0.579282,0.638735,0.625506,0.676936,0.624045,0.741691,0.651541,...,0.726567,0.506784,0.831173,0.551411,0.811718,0.601999,0.81539,0.556134,0.750673,1


In [33]:
# Assuming 'df' is your DataFrame
df_filtered = df_features[['frame', 'NOSE_x', 'NOSE_y', 'is_falling']]

In [34]:
df_filtered.head(10)

,frame,NOSE_x,NOSE_y,is_falling
0,1,0.282740,0.497902,0
1,2,0.289449,0.490814,0
2,3,0.300938,0.483526,0
3,4,0.313169,0.482957,0
4,5,0.322819,0.482779,0
5,6,0.330255,0.482343,0
6,7,0.335796,0.482512,0
7,8,0.336645,0.484410,0
8,9,0.336904,0.489506,0
9,10,0.339928,0.496704,0


In [35]:
def generate_basic_input(features, frames):
  df_features = pd.read_csv(features)
  with open(example_frames, 'r') as file:
    data = file.readlines()
  frame1 = int(data[0].strip())  # First frame
  frame2 = int(data[1].strip())  # Second frame
  df_features['is_falling'] = df_features['frame'].apply(lambda x: 1 if x in range(frame1, frame2 + 1) else 0)

  return df_features

In [36]:
basic_df = generate_basic_input(example_features, example_frames)
basic_df.shape

(144, 28)

In [37]:
def create_lookback_window_df(df, feature_columns, lookback_window):
    """
    Creates a lookback window DataFrame for specified features in a DataFrame.

    Parameters:
    - df: DataFrame containing the features and frames.
    - feature_columns: List of feature column names to apply the lookback window on.
    - lookback_window: Size of the lookback window (number of previous frames to consider).

    Returns:
    - lookback_df: A new DataFrame with the lookback window for the specified features.
    """

    lookback_df = pd.DataFrame()

    for i in range(lookback_window):
      for feature in feature_columns:
            lookback_df[f'{feature}_t-{i+1}'] = df[feature].shift(i+1)

    for feature in feature_columns:
        lookback_df[f'{feature}_t'] = df[feature]

    lookback_df.dropna(inplace=True)

    lookback_df.reset_index(drop=True, inplace=True)

    lookback_df['frame'] = df['frame'].iloc[lookback_window:].reset_index(drop=True)
    lookback_df['is_falling'] = df['is_falling'].iloc[lookback_window:].reset_index(drop=True)

    return lookback_df

In [38]:
lookback_window_size = 10
features = ['NOSE_x', 'NOSE_y']

In [39]:
lstm_input = create_lookback_window_df(basic_df, features, lookback_window_size)
lstm_input.head()

,NOSE_x_t-1,NOSE_y_t-1,NOSE_x_t-2,NOSE_y_t-2,NOSE_x_t-3,NOSE_y_t-3,NOSE_x_t-4,NOSE_y_t-4,NOSE_x_t-5,NOSE_y_t-5,...,NOSE_x_t-8,NOSE_y_t-8,NOSE_x_t-9,NOSE_y_t-9,NOSE_x_t-10,NOSE_y_t-10,NOSE_x_t,NOSE_y_t,frame,is_falling
0,0.339928,0.496704,0.336904,0.489506,0.336645,0.484410,0.335796,0.482512,0.330255,0.482343,...,0.300938,0.483526,0.289449,0.490814,0.282740,0.497902,0.339317,0.500805,11,0
1,0.339317,0.500805,0.339928,0.496704,0.336904,0.489506,0.336645,0.484410,0.335796,0.482512,...,0.313169,0.482957,0.300938,0.483526,0.289449,0.490814,0.340141,0.504424,12,0
2,0.340141,0.504424,0.339317,0.500805,0.339928,0.496704,0.336904,0.489506,0.336645,0.484410,...,0.322819,0.482779,0.313169,0.482957,0.300938,0.483526,0.343814,0.505691,13,0
3,0.343814,0.505691,0.340141,0.504424,0.339317,0.500805,0.339928,0.496704,0.336904,0.489506,...,0.330255,0.482343,0.322819,0.482779,0.313169,0.482957,0.344405,0.505821,14,0
4,0.344405,0.505821,0.343814,0.505691,0.340141,0.504424,0.339317,0.500805,0.339928,0.496704,...,0.335796,0.482512,0.330255,0.482343,0.322819,0.482779,0.343502,0.501333,15,0


In [40]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data

In [41]:
# Define the LSTM class
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()

        # Initialize the LSTM layer
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)

        # Fully connected layer
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)  # lstm_out shape: (batch_size, sequence_length, hidden_size)

        # We only need the output from the last time step
        lstm_out = lstm_out[:, -1, :]

        # Pass through the fully connected layer to get the final output
        out = self.fc(lstm_out)

        return out


In [42]:
feature_list = [col for col in lstm_input.columns if 'NOSE_x' in col or 'NOSE_y' in col]
print(feature_list)

['NOSE_x_t-1', 'NOSE_y_t-1', 'NOSE_x_t-2', 'NOSE_y_t-2', 'NOSE_x_t-3', 'NOSE_y_t-3', 'NOSE_x_t-4', 'NOSE_y_t-4', 'NOSE_x_t-5', 'NOSE_y_t-5', 'NOSE_x_t-6', 'NOSE_y_t-6', 'NOSE_x_t-7', 'NOSE_y_t-7', 'NOSE_x_t-8', 'NOSE_y_t-8', 'NOSE_x_t-9', 'NOSE_y_t-9', 'NOSE_x_t-10', 'NOSE_y_t-10', 'NOSE_x_t', 'NOSE_y_t']


In [43]:
def prepare_lstm_input(df, frame_count, feature_columns, num_features, sequence_length):
    """
    Converts a lookback window DataFrame into a 3D tensor suitable for LSTM input.

    Parameters:
    - df: DataFrame containing the lookback features for LSTM input.
    - feature_columns: List of feature column names to include (e.g., 'NOSE_x_t-10' to 'NOSE_x_t', 'NOSE_y_t-10' to 'NOSE_y_t').
    - sequence_length: Number of time steps in the lookback window (e.g., 10 previous frames plus the current one).

    Returns:
    - X: PyTorch tensor of shape (num_samples, sequence_length+1, num_features)
    - y: PyTorch tensor of the target labels (e.g., is_falling)
    """

    # Step 1: Extract the feature columns and convert to NumPy array
    data = df[feature_columns].values

    data_reshaped = reshaped_data = np.reshape(data, (frame_count, sequence_length, num_features))

    y = df['is_falling'].values  # Extract 1 target per sequence

    # Step 7: Convert to PyTorch tensors
    X = torch.tensor(data_reshaped, dtype=torch.float32)
    y = torch.tensor(y, dtype=torch.float32)

    return X, y

# Example usage:
sequence_length = 11  # Lookback window
feature_list = [col for col in lstm_input.columns if 'NOSE_x' in col or 'NOSE_y' in col]
num_features = 2
frame_count = 134

# Convert DataFrame to LSTM input format
X, y = prepare_lstm_input(lstm_input, frame_count, feature_list, num_features, sequence_length)

# Check the shapes
print("X shape:", X.shape)  # Expected shape: (num_samples, sequence_length+1, num_features)
print("y shape:", y.shape)  # Expected shape: (num_samples,)



X shape: torch.Size([134, 11, 2])
y shape: torch.Size([134])


In [44]:
print(X[1])

tensor([[0.3393, 0.5008],
        [0.3399, 0.4967],
        [0.3369, 0.4895],
        [0.3366, 0.4844],
        [0.3358, 0.4825],
        [0.3303, 0.4823],
        [0.3228, 0.4828],
        [0.3132, 0.4830],
        [0.3009, 0.4835],
        [0.2894, 0.4908],
        [0.3401, 0.5044]])


In [45]:
lstm_input[feature_list].head()

,NOSE_x_t-1,NOSE_y_t-1,NOSE_x_t-2,NOSE_y_t-2,NOSE_x_t-3,NOSE_y_t-3,NOSE_x_t-4,NOSE_y_t-4,NOSE_x_t-5,NOSE_y_t-5,...,NOSE_x_t-7,NOSE_y_t-7,NOSE_x_t-8,NOSE_y_t-8,NOSE_x_t-9,NOSE_y_t-9,NOSE_x_t-10,NOSE_y_t-10,NOSE_x_t,NOSE_y_t
0,0.339928,0.496704,0.336904,0.489506,0.336645,0.484410,0.335796,0.482512,0.330255,0.482343,...,0.313169,0.482957,0.300938,0.483526,0.289449,0.490814,0.282740,0.497902,0.339317,0.500805
1,0.339317,0.500805,0.339928,0.496704,0.336904,0.489506,0.336645,0.484410,0.335796,0.482512,...,0.322819,0.482779,0.313169,0.482957,0.300938,0.483526,0.289449,0.490814,0.340141,0.504424
2,0.340141,0.504424,0.339317,0.500805,0.339928,0.496704,0.336904,0.489506,0.336645,0.484410,...,0.330255,0.482343,0.322819,0.482779,0.313169,0.482957,0.300938,0.483526,0.343814,0.505691
3,0.343814,0.505691,0.340141,0.504424,0.339317,0.500805,0.339928,0.496704,0.336904,0.489506,...,0.335796,0.482512,0.330255,0.482343,0.322819,0.482779,0.313169,0.482957,0.344405,0.505821
4,0.344405,0.505821,0.343814,0.505691,0.340141,0.504424,0.339317,0.500805,0.339928,0.496704,...,0.336645,0.484410,0.335796,0.482512,0.330255,0.482343,0.322819,0.482779,0.343502,0.501333
